In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

from nltk import word_tokenize
import seaborn as sns

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn import metrics

In [ ]:
df = pd.read_csv('Final_augmented.csv')

In [ ]:
df.head()

In [ ]:
print(df['text'].apply(lambda x: len(x.split(' '))).sum())

In [ ]:
special_character_remover = re.compile('[/(){}\[\]\|@,;:]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = nltk.corpus.stopwords.words('french')

In [ ]:
def clean_text(text):
    #text = text.lower()
    text = special_character_remover.sub(' ', text)
    text = extra_symbol_remover.sub('', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    
    return text
    
df['text'] = df['text'].apply(clean_text)

In [ ]:
df.head()

In [ ]:
print(df['text'].apply(lambda x: len(x.split(' '))).sum())

In [ ]:
df = df.drop_duplicates(subset='text', keep="last")

In [ ]:
print(df['text'].apply(lambda x: len(x.split(' '))).sum())

In [ ]:
df.head()

In [ ]:
df=df.rename(columns={"label2": "label"})

## Split the train and test

In [ ]:
from sklearn.model_selection import train_test_split
X = df.text
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

## Apply Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")

In [ ]:
print(classification_report(y_test, y_pred))

## Applying Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB


naivebayes = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
naivebayes.fit(X_train, y_train)

y_pred = naivebayes.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

In [ ]:
print(classification_report(y_test, y_pred))

## Confusion Matrix

In [ ]:
# Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix \n', cf_matrix)

## Applying Xgboost Classifier

In [ ]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
text = "Eclairage de la gaine , à compléter rappel : 2"
cleaned_text = clean_text(text)
print(cleaned_text)

In [ ]:
prediction = lr.predict([cleaned_text])
print(prediction)

In [ ]:
df_original = pd.read_csv('Clean_dataset.csv')

In [ ]:
df_original.head()

In [ ]:
#Create a labels map
X = df_original["text"].tolist()
y = pd.get_dummies(df_original['label'])
mapping = {i: name for i, name in enumerate(y.columns)}
mapping

In [ ]:
print(prediction[0])

In [ ]:
mapping[prediction[0]]